### Data Cleaning

---

### Imports
Importing all Python Libararies that will be of of use in this project:

In [134]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

%matplotlib inline

Reading the CSV dataset into the our system as a dataframe that can be examined.

In [135]:
df = pd.read_csv("./datasets/train.csv")

Looking at the overall data with shape, describe, dtypes to get a better understanding of the current state of data:

In [136]:
df.shape

(2051, 81)

In [137]:
df.info

<bound method DataFrame.info of         Id        PID  MS SubClass MS Zoning  Lot Frontage  Lot Area Street  \
0      109  533352170           60        RL           NaN     13517   Pave   
1      544  531379050           60        RL          43.0     11492   Pave   
2      153  535304180           20        RL          68.0      7922   Pave   
3      318  916386060           60        RL          73.0      9802   Pave   
4      255  906425045           50        RL          82.0     14235   Pave   
...    ...        ...          ...       ...           ...       ...    ...   
2046  1587  921126030           20        RL          79.0     11449   Pave   
2047   785  905377130           30        RL           NaN     12342   Pave   
2048   916  909253010           50        RL          57.0      7558   Pave   
2049   639  535179160           20        RL          80.0     10400   Pave   
2050    10  527162130           60        RL          60.0      7500   Pave   

     Alley Lot Shap

In [138]:
df.dtypes

Id                int64
PID               int64
MS SubClass       int64
MS Zoning        object
Lot Frontage    float64
                 ...   
Misc Val          int64
Mo Sold           int64
Yr Sold           int64
Sale Type        object
SalePrice         int64
Length: 81, dtype: object

### Dealing with Null values

Before making any changes to the DataFrame, I am changing all column titles to be the same syntax as lowercase.

In [139]:
df.columns = [column_name.lower().replace(' ','_') for column_name in df.columns]

Looking for all the null values in the data

In [140]:
df.isnull().sum().sum()

9822

Singling out the specific columns with nulls to be dealt with

In [141]:
null_columns = df.columns[df.isna().any()].tolist()
null_columns

['lot_frontage',
 'alley',
 'mas_vnr_type',
 'mas_vnr_area',
 'bsmt_qual',
 'bsmt_cond',
 'bsmt_exposure',
 'bsmtfin_type_1',
 'bsmtfin_sf_1',
 'bsmtfin_type_2',
 'bsmtfin_sf_2',
 'bsmt_unf_sf',
 'total_bsmt_sf',
 'bsmt_full_bath',
 'bsmt_half_bath',
 'fireplace_qu',
 'garage_type',
 'garage_yr_blt',
 'garage_finish',
 'garage_cars',
 'garage_area',
 'garage_qual',
 'garage_cond',
 'pool_qc',
 'fence',
 'misc_feature']

### Creating a function to clean my Data

After looking at the data dictionary, here were the following conclusions I made about the nulls that I will apply to my data:
- for categorical variables, we don't want to drop nulls as having none of something is also valuable information.
- for numerical variables, there is no surefire way to know if a missing value is 0 or just missing data.

---

Using the above assumption the following function was written to combat most of the data cleaning:

Changes all string nulls to 'none'. For numerical columns the nans will be imputed with its average value for the neighborhood that house resides in. **IF** a relating categorical column says that value doesnt exist, we know that that missing value can be imputed with 0 instead. _(e.g. If a house has 'no pool' we populate pool area with 0. If there is a pool but we don't know the pool area, we impute the nan with the avg pool area of that house's neighborhood's pools)_

In [142]:
#imputing null values with the avg value for the neighborhood of that specific house. 
#IF it is a categorical variable and another column indicates that the house doesn't have it (e.g. no Basement), the value is imputed with 0.0
def impute_avg_of_neighborhood(df):
    
    #for later use in the function of math.isnan(x)
    #Return True if x is a NaN (not a number), and False otherwise.
    import math
    
    #returns a list of all the columns with any null values
    null_columns = df.columns[df.isna().any()].tolist()
    
    for i in null_columns:
        if df[i].dtype == 'object':
            df[i].replace(np.nan, 'none', inplace = True)
    
    for i in null_columns:
        if df[i].dtype == 'float64':
            
            #getting indexes of all null values in that column
            null_indexes = df[df[i].isnull()].index
            
            #creating a dictionary to store mean values of each neighborhood per the column variable
            key = list(df[['neighborhood', i]].groupby('neighborhood').mean().index)
            value = list(df[['neighborhood', i]].groupby('neighborhood').mean()[i].values)
            mean_dict = dict(zip(key, value))
            
            #of the null columns, if a relating categorical column says "none", then impute with 0.0's 
            for x in null_indexes:
                if 'mas_vnr' in i:
                    if df.at[ x,'mas_vnr_type'] == 'none':
                        df.at[x, i] = 0.0
                elif 'bsmt' in i:  
                    if df.at[ x,'bsmt_qual'] == 'none':
                        df.at[x, i] = 0.0
                elif 'pool' in i:  
                    if df.at[ x,'pool_qc'] == 'none':
                        df.at[x, i] = 0.0                 
                elif 'garage' in i:  
                    if df.at[ x,'garage_type'] == 'none':
                        df.at[x, i] = 0.0
                        
                    else:
                        mean_dict_key = df.at[x,'neighborhood']
                    
                        if math.isnan(mean_dict[mean_dict_key]):
                    
                            df.at[x, i] = 0
                        
                        else:
                            df.at[x, i] = mean_dict[mean_dict_key]
               
            #otherwise impute with the avg value of the house's neighborhood
                else:
                    mean_dict_key = df.at[x,'neighborhood']
                    
                    if math.isnan(mean_dict[mean_dict_key]):
                    
                        df.at[x, i] = 0
                        
                    else:
                        df.at[x, i] = mean_dict[mean_dict_key]
                    
    return df

In [143]:
impute_avg_of_neighborhood(df)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,72.774648,13517,Pave,none,IR1,Lvl,...,0,0,none,none,none,0,3,2010,WD,130500
1,544,531379050,60,RL,43.000000,11492,Pave,none,IR1,Lvl,...,0,0,none,none,none,0,4,2009,WD,220000
2,153,535304180,20,RL,68.000000,7922,Pave,none,Reg,Lvl,...,0,0,none,none,none,0,1,2010,WD,109000
3,318,916386060,60,RL,73.000000,9802,Pave,none,Reg,Lvl,...,0,0,none,none,none,0,4,2010,WD,174000
4,255,906425045,50,RL,82.000000,14235,Pave,none,IR1,Lvl,...,0,0,none,none,none,0,3,2010,WD,138500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1587,921126030,20,RL,79.000000,11449,Pave,none,IR1,HLS,...,0,0,none,none,none,0,1,2008,WD,298751
2047,785,905377130,30,RL,66.930233,12342,Pave,none,IR1,Lvl,...,0,0,none,none,none,0,3,2009,WD,82500
2048,916,909253010,50,RL,57.000000,7558,Pave,none,Reg,Bnk,...,0,0,none,none,none,0,3,2009,WD,177000
2049,639,535179160,20,RL,80.000000,10400,Pave,none,Reg,Lvl,...,0,0,none,none,none,0,11,2009,WD,144000


Confirming that there are no more nulls is the cleaned dataframe.

In [144]:
df.isnull().sum().sort_values(ascending = True)

id                0
fireplaces        0
functional        0
totrms_abvgrd     0
kitchen_qual      0
                 ..
roof_matl         0
roof_style        0
year_remod/add    0
total_bsmt_sf     0
saleprice         0
Length: 81, dtype: int64

In [145]:
import pickle
pickle.dump(df, open('./pickles/Clean_DataFrame.pkl', 'wb'))

### Dropping Outliers based on basic EDA (test data specific)

*See EDA notebbook for explanations on these rows dropped from the training data.

In [146]:
df[df['gr_liv_area'] > 5000]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
960,1499,908154235,60,RL,313.0,63887,Pave,none,IR3,Bnk,...,0,480,Gd,none,none,0,1,2008,New,160000
1885,2181,908154195,20,RL,128.0,39290,Pave,none,IR1,Bnk,...,0,0,none,none,Elev,17000,10,2007,New,183850


In [147]:
#drop outliers which were sales above 5000 sq ft
df.drop(index = [960, 1885], inplace= True)

In [148]:
df[df['1st_flr_sf'] > 3500]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
616,1498,908154080,20,RL,123.0,47007,Pave,none,IR1,Lvl,...,0,0,none,none,none,0,7,2008,WD,284700


In [149]:
#dropping outlier  of 1st floor sqft above 3500
df.drop(index = 616, inplace= True)

In [150]:
pickle.dump(df, open('./pickles/Clean_DataFrame_v2.pkl', 'wb'))

### Organizing important ordinal variables that will be included in my Model

In [151]:
bsmt_qual_dict = dict(zip(['none', 'Po', 'Fa', 'TA','Gd', 'Ex'], range(6)))
df['bsmt_qual'] = df['bsmt_qual'].apply(lambda x:bsmt_qual_dict[x])

kitchen_qual_dict = dict(zip(['Po', 'Fa', 'TA','Gd', 'Ex'], range(1, 6)))
df['kitchen_qual'] = df['kitchen_qual'].apply(lambda x:kitchen_qual_dict[x])

exter_qual_dict = dict(zip(['Po', 'Fa', 'TA','Gd', 'Ex'], range(1, 6)))
df['exter_qual'] = df['exter_qual'].apply(lambda x:exter_qual_dict[x])

heating_qc_dict = dict(zip(['Po', 'Fa', 'TA','Gd', 'Ex'], range(1, 6)))
df['heating_qc'] = df['heating_qc'].apply(lambda x:heating_qc_dict[x])

fireplace_qu_dict = dict(zip(['none', 'Po', 'Fa', 'TA','Gd', 'Ex'], range(6)))
df['fireplace_qu'] = df['fireplace_qu'].apply(lambda x:fireplace_qu_dict[x])

garage_finish_dict = dict(zip(['none', 'Unf', 'RFn', 'Fin'], range(4)))
df['garage_finish'] = df['garage_finish'].apply(lambda x:garage_finish_dict[x])


### Dummying out categorical columns I want to include in my Model

In [152]:
df = pd.get_dummies(df, columns = ['foundation', 'bsmtfin_type_1', 'neighborhood', 'mas_vnr_type'] )
df.columns = [column_name.lower().replace(' ','_') for column_name in df.columns]

In [153]:
df['overall_livingarea'] = df['overall_qual']* df['gr_liv_area']

In [154]:
pickle.dump(df, open('./pickles/Clean_DataFrame_v3.pkl', 'wb'))